In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [4]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold']


In [5]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition']

cat_att = ['LotShape', 'LandContour', 'Neighborhood', 'BldgType', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
           'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']

bsmt_cat_att = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1']
for a in bsmt_cat_att:
    train[a].fillna('NB', inplace=True)
    test[a].fillna('NB', inplace=True)
    
# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['LotShape', 'LandContour', 'Neighborhood', 'BldgType', 'HouseStyle', 'RoofStyle', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']


In [6]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(median, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(median, inplace=True)

In [7]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
GrLivArea        0.730255
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
GarageYrBlt      0.495794
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
WoodDeckSF       0.343269
LotFrontage      0.335292
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF    -0.037963
MSSubClass  

In [9]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 44 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [11]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 28)
(1459, 28)
(1460, 128)
(1459, 128)


In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=350, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.0010380244879687696

In [13]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [14]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

138.8239561265225

In [15]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [16]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.21291380453716174, 'OverallQual'),
 (0.13924006417756377, 'GrLivArea'),
 (0.07774348427078645, 'YearBuilt'),
 (0.05009751007392967, 'TA'),
 (0.04922647264647009, 'TotalBsmtSF'),
 (0.047486137211906206, 'GarageCars'),
 (0.04484399296531045, 'GarageArea'),
 (0.03997052211576378, 'FullBath'),
 (0.03916783633451459, '1stFlrSF'),
 (0.03765085902211672, 'GarageYrBlt'),
 (0.021440261496741015, 'LotArea'),
 (0.020607373570149985, 'Fireplaces'),
 (0.01936579004788021, 'BsmtFinSF1'),
 (0.017397413629657073, '2ndFlrSF'),
 (0.013964436409320177, 'YearRemodAdd'),
 (0.010911020785489653, 'LotFrontage'),
 (0.0107440442849584, 'OverallCond'),
 (0.010240100690355037, 'Y'),
 (0.009427904207625945, 'Ex'),
 (0.008787422891256828, 'N'),
 (0.008692816558520318, 'TotRmsAbvGrd'),
 (0.00707634632720354, 'BsmtUnfSF'),
 (0.006071379983271603, 'OpenPorchSF'),
 (0.005951250572534313, 'PConc'),
 (0.005672957820804968, 'Gd'),
 (0.004868605605401451, 'MSSubClass'),
 (0.0046943407486131385, 'BedroomAbvGr'),
 (0.00

In [17]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=200, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [0.13151602 0.11976503 0.13245626 0.16546516 0.16333717 0.1146201
 0.12820989 0.11542195 0.12496702 0.13771727]
mean: 0.1333475884440734
std: 0.017050862411555293
count    10.000000
mean      0.133348
std       0.017973
min       0.114620
25%       0.121066
50%       0.129863
75%       0.136402
max       0.165465
dtype: float64


In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
#     {'n_estimators': [100, 200, 300, 350, 400, 450], 'max_features': [16, 32, 64]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 200, 300, 350, 400, 450], 
     'max_features': [16, 32, 64]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_full_prepared, train_y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'bootstrap': [False], 'max_feat

In [19]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 32, 'n_estimators': 350}

In [20]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=32, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=350,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [21]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.1365325291773575 {'bootstrap': False, 'max_features': 16, 'n_estimators': 100}
0.13554563495239574 {'bootstrap': False, 'max_features': 16, 'n_estimators': 200}
0.13513768345283997 {'bootstrap': False, 'max_features': 16, 'n_estimators': 300}
0.13513013329654988 {'bootstrap': False, 'max_features': 16, 'n_estimators': 350}
0.1352157620506805 {'bootstrap': False, 'max_features': 16, 'n_estimators': 400}
0.13512899203788603 {'bootstrap': False, 'max_features': 16, 'n_estimators': 450}
0.13435808340986752 {'bootstrap': False, 'max_features': 32, 'n_estimators': 100}
0.13443329666726084 {'bootstrap': False, 'max_features': 32, 'n_estimators': 200}
0.13416966597876986 {'bootstrap': False, 'max_features': 32, 'n_estimators': 300}
0.1341140373035052 {'bootstrap': False, 'max_features': 32, 'n_estimators': 350}
0.1343312910648253 {'bootstrap': False, 'max_features': 32, 'n_estimators': 400}
0.1344320149776575 {'bootstrap': False, 'max_features': 32, 'n_estimators': 450}
0.13587238015272834 {

In [22]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([4.86860561e-03, 1.09110208e-02, 2.14402615e-02, 2.12913805e-01,
       1.07440443e-02, 7.77434843e-02, 1.39644364e-02, 4.27999268e-03,
       1.93657900e-02, 7.07634633e-03, 4.92264726e-02, 3.91678363e-02,
       1.73974136e-02, 1.39240064e-01, 2.27507988e-03, 3.99705221e-02,
       2.54778652e-03, 4.69434075e-03, 2.21726636e-03, 8.69281656e-03,
       2.06073736e-02, 3.76508590e-02, 4.74861372e-02, 4.48439930e-02,
       3.65247097e-03, 6.07137998e-03, 2.25190648e-03, 2.42481456e-03,
       9.00940642e-04, 2.55170226e-04, 1.22632311e-04, 1.19896654e-03,
       1.22119056e-03, 3.04631320e-04, 4.80854426e-04, 8.11712008e-04,
       4.60392574e-05, 4.27986213e-06, 1.08714523e-04, 3.46632137e-04,
       4.00805505e-04, 2.85883345e-04, 1.51835830e-03, 1.46933783e-03,
       1.88654550e-04, 1.00081292e-03, 2.01529178e-04, 1.55402052e-04,
       5.82638656e-04, 3.47300023e-05, 2.00808094e-04, 3.84599189e-04,
       2.47605560e-04, 1.64310908e-03, 1.28693087e-04, 4.92910283e-04,
      

In [23]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.21291380453716174, 'OverallQual'),
 (0.13924006417756377, 'GrLivArea'),
 (0.07774348427078645, 'YearBuilt'),
 (0.05009751007392967, 'TA'),
 (0.04922647264647009, 'TotalBsmtSF'),
 (0.047486137211906206, 'GarageCars'),
 (0.04484399296531045, 'GarageArea'),
 (0.03997052211576378, 'FullBath'),
 (0.03916783633451459, '1stFlrSF'),
 (0.03765085902211672, 'GarageYrBlt'),
 (0.021440261496741015, 'LotArea'),
 (0.020607373570149985, 'Fireplaces'),
 (0.01936579004788021, 'BsmtFinSF1'),
 (0.017397413629657073, '2ndFlrSF'),
 (0.013964436409320177, 'YearRemodAdd'),
 (0.010911020785489653, 'LotFrontage'),
 (0.0107440442849584, 'OverallCond'),
 (0.010240100690355037, 'Y'),
 (0.009427904207625945, 'Ex'),
 (0.008787422891256828, 'N'),
 (0.008692816558520318, 'TotRmsAbvGrd'),
 (0.00707634632720354, 'BsmtUnfSF'),
 (0.006071379983271603, 'OpenPorchSF'),
 (0.005951250572534313, 'PConc'),
 (0.005672957820804968, 'Gd'),
 (0.004868605605401451, 'MSSubClass'),
 (0.0046943407486131385, 'BedroomAbvGr'),
 (0.00

In [24]:
# y_pred = forest_reg.predict(test_full_prepared)
y_pred = grid_search.predict(test_full_prepared)
y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [25]:
sub.to_csv("data/submission_script28_anova_1st_attempt.csv", index=False)

In [26]:
sub.shape

(1459, 2)